# 8 Puzzle solver
* Parsa KamaliPour - 97149081
* In this repository we're going to solve this puzzle using $ A^* $ and $ IDA $

#### imports:

In [1]:
import pandas as pd
import numpy as np

#### Test case 1:

In [2]:
input_puzzle = [
    [1, 2, 3],
    [4, 0, 5],
    [7, 8, 6]
]

print('Input: ')
print(pd.DataFrame(input_puzzle))
print()

desired_output = [
    [1, 2, 3],
    [4, 5, 0],
    [7, 8, 6]
]

print('Desired Output:')
print(pd.DataFrame(desired_output))

Input: 
   0  1  2
0  1  2  3
1  4  0  5
2  7  8  6

Desired Output:
   0  1  2
0  1  2  3
1  4  5  0
2  7  8  6


### The node class:

In [3]:
class Node:

    def __init__(self, parent, g_cost, row, col, val, h_cost, omega):

        self.parent = parent
        self.g_cost = g_cost
        self.row = row
        self.col = col
        self.val = val
        self.h_cost = h_cost
        self.f_function = (2 - omega) * self.g_cost + omega * self.h_cost